In [1]:
import os
#os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"   # see issue #152z
#os.environ["CUDA_VISIBLE_DEVICES"] = ""

# Input data files are available in the "../input/" directory.
# For example, running this will list the files in the input directory

import numpy as np   # linear algebra
import sklearn
import keras
import matplotlib.pyplot as plt
from sklearn.preprocessing import minmax_scale
print(np.__version__)



Using TensorFlow backend.


1.17.4


In [2]:
train_data = np.loadtxt('../input/train.csv', str, delimiter = ',', skiprows=1)
train_value = np.loadtxt('../input/train.csv', delimiter = ',', skiprows=1, usecols=[1])
leng_all = len(train_data)

In [3]:
train_value.reshape(leng_all,1)
train_value.shape

(2476315,)

In [4]:
id_list = np.unique(train_data[:,3])
timeseries_all = []


for kpi_id in id_list:
    mask = (train_data[:, 3]==kpi_id)
    timeseries_all.append(train_value[mask])
print(timeseries_all)


[array([1.90163934, 1.78688526, 2.        , ..., 2.61403513, 2.73684216,
       2.4912281 ]), array([ 0.83281839, -0.04328661,  0.06622652, ...,  0.50427902,
        0.61379214,  0.50427902]), array([0.19757504, 0.14987727, 0.07889324, ..., 0.08827026, 0.09042697,
       0.07923706]), array([9.13373376e+08, 9.01925824e+08, 9.15506240e+08, ...,
       9.60516800e+08, 9.61848768e+08, 9.62367296e+08]), array([2.25      , 3.1500001 , 3.04999995, ..., 5.44999981, 6.80000019,
       5.69999981]), array([1204.625 , 1182.25  , 1155.1875, ..., 1371.375 , 1381.0625,
       1376.9375]), array([ 628.,  766.,  858., ..., 1668., 1698., 1670.]), array([-1.04808758, -1.01906734, -0.96832112, ...,  1.03450159,
        0.77182508,  1.00960628]), array([1839., 1842., 1947., ..., 1497., 1532., 1587.]), array([0.10381233, 0.10144436, 0.10412569, ..., 0.13573759, 0.13261388,
       0.12666015]), array([0.09563687, 0.09369217, 0.1014476 , ..., 0.13831563, 0.14613896,
       0.12443224]), array([2109., 2064.,

In [5]:
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
import random

timeseries_num = len(timeseries_all)
print(timeseries_num)

26


In [9]:
train_label = np.loadtxt('../input/train.csv', delimiter = ',', skiprows=1, usecols=[2])
train_label

array([0., 0., 0., ..., 1., 1., 1.])

In [10]:
timeseries_label = []
for kpi_id in id_list:
    mask = (train_data[:, 3]==kpi_id)
    timeseries_label.append(train_label[mask])
print(timeseries_label)

[array([0., 0., 0., ..., 0., 0., 0.]), array([0., 0., 0., ..., 0., 0., 0.]), array([0., 0., 0., ..., 0., 0., 0.]), array([0., 0., 0., ..., 0., 0., 0.]), array([0., 0., 0., ..., 0., 0., 0.]), array([0., 0., 0., ..., 0., 0., 0.]), array([0., 0., 0., ..., 0., 0., 0.]), array([0., 0., 0., ..., 0., 0., 0.]), array([0., 0., 0., ..., 0., 0., 0.]), array([0., 0., 0., ..., 0., 0., 0.]), array([0., 0., 0., ..., 0., 0., 0.]), array([0., 0., 0., ..., 0., 0., 0.]), array([0., 0., 0., ..., 1., 1., 1.]), array([0., 0., 0., ..., 0., 0., 0.]), array([0., 0., 0., ..., 0., 0., 0.]), array([0., 0., 0., ..., 0., 0., 0.]), array([0., 0., 0., ..., 0., 0., 0.]), array([0., 0., 0., ..., 0., 0., 0.]), array([0., 0., 0., ..., 0., 0., 0.]), array([0., 0., 0., ..., 0., 0., 0.]), array([0., 0., 0., ..., 0., 0., 0.]), array([0., 0., 0., ..., 0., 0., 0.]), array([0., 0., 0., ..., 0., 0., 0.]), array([0., 0., 0., ..., 0., 0., 0.]), array([0., 0., 0., ..., 0., 0., 0.]), array([0., 0., 0., ..., 0., 0., 0.])]


In [13]:
assert(len(timeseries_all) == len(timeseries_label))
for i in range(timeseries_num):
    assert(timeseries_all[i].shape == timeseries_label[i].shape)

完成了 timeseries_all (26 个 kpi 值的时间序列所组成的列表）以及 timeseries_label（26 个 kpi 标注序列所组成的列表）的提取

接下来开始数据的预处理

（1）异常样本的过采样
（2）特征提取




In [26]:
from collections import Counter
oversample_ratio_list = []
for i in range(timeseries_num):
    temp = [0, 0]
    c = Counter(timeseries_label[i])
    for key,value in c.items():
        temp[int(key)] = value
    oversample_ratio = round((temp[0] - temp[1])/temp[1])
    oversample_ratio_list.append(oversample_ratio)
print(oversample_ratio_list)

[10, 108, 82, 540, 14, 11, 151, 2060, 131, 974, 99, 345, 19, 2194, 244, 46, 897, 20, 360, 4435, 251, 685, 1036, 134, 15, 46]


In [28]:
timeseries_oversample = []
timeseries_label_oversample = []
for i in range(timeseries_num):
    oversample_temp = []
    oversample_label_temp = []
    find_before = False
    before_anomaly_loc = 0
    for j in range(len(timeseries_all[i])):
        if int(timeseries_label[i][j]) == 1:
            ratio = oversample_ratio_list[i]
            if find_before:
                for k in range(ratio):
                    average_random = random.random()
                    oversample_temp.append(timeseries_all[i][j]*average_random + timeseries_all[i][before_anomaly_loc]*(1-average_random))
                    oversample_label_temp.append(1.0)
            else:
                for k in range(ratio):
                    oversample_temp.append(timeseries_all[i][j] * (1 + random.randint(-5,5)/100))
                    oversample_label_temp.append(1.0)
                find_before = True
                before_anomaly_loc = j
        else:
            oversample_temp.append(timeseries_all[i][j])
            oversample_label_temp.append(timeseries_label[i][j])
            find_before = False
    timeseries_oversample.append(np.asarray(oversample_temp))
    timeseries_label_oversample.append(np.asarray(oversample_label_temp))
    assert(len(oversample_label_temp) == len(oversample_temp))
for i in range(timeseries_num):
    print(Counter(timeseries_label_oversample[i]))
assert(len(timeseries_oversample) == len(timeseries_all))

Counter({0.0: 118012, 1.0: 105500})
Counter({0.0: 8704, 1.0: 8640})
Counter({0.0: 10830, 1.0: 10660})
Counter({0.0: 128733, 1.0: 128520})
Counter({0.0: 121265, 1.0: 110082})
Counter({0.0: 119272, 1.0: 105391})
Counter({0.0: 99600, 1.0: 98754})
Counter({0.0: 8244, 1.0: 8240})
Counter({0.0: 146555, 1.0: 145803})
Counter({0.0: 8775, 1.0: 8766})
Counter({0.0: 8697, 1.0: 8613})
Counter({0.0: 147263, 1.0: 146970})
Counter({0.0: 62330, 1.0: 59014})
Counter({0.0: 8780, 1.0: 8776})
Counter({0.0: 128926, 1.0: 128588})
Counter({0.0: 143926, 1.0: 141818})
Counter({0.0: 128470, 1.0: 128271})
Counter({0.0: 62504, 1.0: 58900})
Counter({0.0: 137544, 1.0: 137160})
Counter({0.0: 128650, 1.0: 128615})
Counter({0.0: 147096, 1.0: 146584})
Counter({0.0: 8235, 1.0: 8220})
Counter({0.0: 128543, 1.0: 128464})
Counter({0.0: 146542, 1.0: 145658})
Counter({0.0: 121369, 1.0: 114990})
Counter({0.0: 143950, 1.0: 141404})


In [29]:
import pickle
file = open('timeseries_all.txt','wb')
pickle.dump(timeseries_all,file)
pickle.dump(timeseries_label,file)
file.close()

In [30]:
file = open('timeseries_oversample.txt','wb')
pickle.dump(timeseries_oversample,file)
pickle.dump(timeseries_label_oversample,file)
file.close()

In [34]:
def get_features_from_a_timeseries(timeseries):
    window = [10, 20, 40, 60]
    features = []
    last_value = []
    for i in range(len(window)):
        last_value.append([0,0,0,0,0])
            
    for i in range(len(timeseries)):
        data = []
        for j in range(len(window)):
            w = window[j]
            if i < w - 1:
                mean1 = np.mean(timeseries[0:i+1])
                var1 = np.var(timeseries[0:i+1])
                median1 = np.median(timeseries[0:i+1])
                quarter1 = np.percentile(timeseries[0:i+1],25)
                quarter3 = np.percentile(timeseries[0:i+1],75)
            else:
                mean1 = np.mean(timeseries[i-w+1:i+1])
                var1 = np.var(timeseries[i-w+1:i+1])
                median1 = np.median(timeseries[i-w+1:i+1])
                quarter1 = np.percentile(timeseries[i-w+1:i+1],25)
                quarter3 = np.percentile(timeseries[i-w+1:i+1],75)
            data.append(mean1)
            data.append(var1)
            data.append(median1)
            data.append(quarter1)
            data.append(quarter3)
            temp_value = [mean1,var1,median1,quarter1,quarter3]
            if i == 0:
                for m in range(5):
                    data.append(0)
                    data.append(0)
                    last_value[j][m] = temp_value[m]
            else:
                for m in range(5):
                    data.append(temp_value[m] - last_value[j][m])
                    data.append((temp_value[m] - last_value[j][m]) / (last_value[j][m]+0.00000001))
                    last_value[j][m] = temp_value[m]
        if i == 0:
            diff = 0
            diff_ratio = 0
        else:
            diff = timeseries[i] - timeseries[i-1]
            diff_ratio = (timeseries[i] - timeseries[i-1]) / (timeseries[i-1] + 0.00000001)
        data.append(diff)
        data.append(diff_ratio)
        assert(len(data) == 62)
        features.append(data)
    assert(len(features) == len(timeseries))
    return features
                

In [38]:
timeseries_features = None
scaler_list = [] 
for i in range(timeseries_num):
    features_temp = get_features_from_a_timeseries(timeseries_oversample[i])
    assert(len(features_temp)==len(timeseries_label_oversample[i]))
    scaler_temp = StandardScaler()
    features_temp = scaler_temp.fit_transform(features_temp)
    scaler_list.append(scaler_temp)
    if i==0:
        timeseries_features = features_temp
    else:
        timeseries_features = np.concatenate((timeseries_features, features_temp), axis = 0)
    
print(timeseries_features.shape)
print(len(scaler_list))

(4785216, 62)
26


In [39]:
timeseries_features = timeseries_features.T
file = open('timeseries_features.txt','wb')
pickle.dump(timeseries_features,file)
pickle.dump(scaler_list,file)
file.close()

In [40]:
connect_labels = []
for i in range(timeseries_num):
    connect_labels = connect_labels + list(timeseries_label_oversample[i])
connect_labels = np.array(connect_labels)
print(connect_labels.shape)
print(timeseries_features.shape)

(4785216,)
(62, 4785216)


In [41]:
file = open('connect_labels.txt','wb')
pickle.dump(connect_labels,file)
file.close()

In [42]:
timeseries_features = timeseries_features.T
print(timeseries_features.shape)

(4785216, 62)


In [43]:
# import Keras libraries
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import Activation
from keras.layers.normalization import BatchNormalization
from keras.regularizers import l2

from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score

# fix random seed for reproducibility
np.random.seed(5)

In [44]:
model = Sequential()
model.add(Dense(128, input_dim=62))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.2))

model.add(Dense(128))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.2))

model.add(Dense(1))
model.add(BatchNormalization())
model.add(Activation('sigmoid'))


model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy','binary_accuracy'])





Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [45]:
print(connect_labels)
print(timeseries_features)

[0. 0. 0. ... 0. 0. 0.]
[[-3.18154113e-01 -4.35271286e-01 -3.08292311e-01 ... -3.85405930e-03
  -6.57747447e-06 -8.16809609e-02]
 [-4.07072349e-01 -4.17547752e-01 -3.95611087e-01 ... -2.08978377e+00
  -2.86146831e-01 -3.96273438e-01]
 [-3.26622508e-01 -3.94438829e-01 -3.08292311e-01 ...  5.74467931e+00
   5.31396793e-01  5.40082459e-01]
 ...
 [-9.12215594e-01 -8.26560643e-01 -9.43588039e-01 ... -2.13887348e-01
   2.82205950e-01 -6.53474325e-02]
 [-9.01833340e-01 -8.26891964e-01 -9.18630573e-01 ... -5.39539447e-03
  -9.63642667e-02 -1.29086885e-01]
 [-9.01949559e-01 -8.26885010e-01 -9.19206515e-01 ... -4.04356124e-01
  -7.55890719e-02 -1.26533514e-01]]


In [47]:
import time
print('Keras: start to train DNN!')
start_time = time.time()
history = model.fit(timeseries_features, connect_labels, epochs=100, batch_size=5000, verbose=1)

end_time = time.time()
print('It took %d seconds to train the model!' %(end_time-start_time))

Keras: start to train DNN!



Epoch 1/100





4785216/4785216 [==============================] - 48s 10us/step - loss: 0.4220 - acc: 0.8216 - binary_accuracy: 0.8216
Epoch 2/100
4785216/4785216 [==============================] - 45s 9us/step - loss: 0.3182 - acc: 0.8709 - binary_accuracy: 0.8709
Epoch 3/100
4785216/4785216 [==============================] - 45s 9us/step - loss: 0.2836 - acc: 0.8833 - binary_accuracy: 0.8833
Epoch 4/100
4785216/4785216 [==============================] - 45s 9us/step - loss: 0.2643 - acc: 0.8904 - binary_accuracy: 0.8904
Epoch 5/100
4785216/4785216 [==============================] - 45s 9us/step - loss: 0.2510 - acc: 0.8958 - binary_accuracy: 0.8958
Epoch 6/100
4785216/4785216 [==============================] - 45s 9us/step - loss: 0.2420 - acc: 0.8995 - binary_accuracy: 0.8995
Epoch 7/100
4785216/4785216 [==============================] - 45s 9us/step - loss: 0.2363 - acc: 0.9021 - binary_accuracy: 0.9021
Epoch 8/100
4785216/4785216 [==================

4785216/4785216 [==============================] - 46s 10us/step - loss: 0.1845 - acc: 0.9261 - binary_accuracy: 0.9261
Epoch 51/100
4785216/4785216 [==============================] - 46s 10us/step - loss: 0.1846 - acc: 0.9261 - binary_accuracy: 0.9261
Epoch 52/100
4785216/4785216 [==============================] - 45s 9us/step - loss: 0.1838 - acc: 0.9264 - binary_accuracy: 0.9264
Epoch 53/100
4785216/4785216 [==============================] - 45s 9us/step - loss: 0.1838 - acc: 0.9264 - binary_accuracy: 0.9264
Epoch 54/100
4785216/4785216 [==============================] - 46s 10us/step - loss: 0.1828 - acc: 0.9269 - binary_accuracy: 0.9269
Epoch 55/100
4785216/4785216 [==============================] - 46s 10us/step - loss: 0.1827 - acc: 0.9269 - binary_accuracy: 0.9269
Epoch 56/100
4785216/4785216 [==============================] - 46s 10us/step - loss: 0.1833 - acc: 0.9266 - binary_accuracy: 0.9266
Epoch 57/100
4785216/4785216 [==============================] - 46s 10us/step - loss

In [49]:
file = open('model.txt','wb')
pickle.dump(model,file)
pickle.dump(history,file)
file.close()

模型训练完毕，接下来开始对测试集进行读取，以及特征提取





In [50]:
test_data = np.loadtxt('../input/test.csv', str, delimiter = ',', skiprows=1)
test_value = np.loadtxt('../input/test.csv', delimiter = ',', skiprows=1, usecols=[1])
leng_test = len(test_data)

In [51]:
test_value.reshape(leng_test,1)
test_value.shape

(2345211,)

In [52]:
test_id_list = np.unique(test_data[:,2])
testseries_all = []


for kpi_id in test_id_list:
    mask = (test_data[:, 2]==kpi_id)
    testseries_all.append(test_value[mask])
print(testseries_all)

[array([2.71929836, 2.45614028, 2.33333325, ..., 1.46753252, 1.44155848,
       1.4545455 ]), array([ 0.61379214,  0.72330527,  0.72330527, ...,  0.06622652,
       -0.04328661, -0.26231286]), array([ 0.08339421,  0.07542374,  0.07589259, ..., -0.29456217,
       -0.29134273, -0.28743564]), array([9.63912960e+08, 9.64395008e+08, 1.03828864e+09, ...,
       1.18040589e+09, 1.17810714e+09, 1.17690176e+09]), array([5.        , 5.5       , 5.1500001 , ..., 3.5       , 3.8499999 ,
       2.79999995]), array([1414.8125  , 1343.8125  , 1367.4375  , ..., 1463.705933,
       1487.529419, 1431.411743]), array([1620., 1609., 1610., ..., 1867., 1849., 1816.]), array([0.8444066 , 0.40438471, 0.92587745, ..., 0.12666432, 0.20291227,
       0.21687184]), array([1596., 1491., 1411., ..., 1724., 1656., 1619.]), array([0.13436497, 0.12777471, 0.13799619, ..., 0.10366487, 0.10001522,
       0.10048587]), array([0.13218766, 0.13275537, 0.13674607, ..., 0.09728213, 0.09286509,
       0.08449186]), array([2

In [53]:

testseries_num = len(testseries_all)
print(testseries_num)

26


In [54]:
file = open('testseries_all.txt','wb')
pickle.dump(testseries_all,file)
file.close()

In [55]:
assert(len(test_id_list)==len(id_list))
for i in range(len(test_id_list)):
    assert(test_id_list[i]==id_list[i]) 

In [56]:
testseries_features = None
for i in range(testseries_num):
    features_temp = get_features_from_a_timeseries(testseries_all[i])
    assert(len(features_temp)==len(testseries_all[i]))
    features_temp = scaler_list[i].transform(features_temp)
    if i==0:
        testseries_features = features_temp
    else:
        testseries_features = np.concatenate((testseries_features, features_temp), axis = 0)
    
print(timeseries_features.shape)

(4785216, 62)


In [57]:
print(testseries_features.shape)

(2345211, 62)


In [58]:
print(testseries_features)

[[ 9.48986834e-01 -4.35271286e-01  9.36055335e-01 ... -3.85405930e-03
  -6.57747447e-06 -8.16809609e-02]
 [ 7.45076417e-01 -3.42064512e-01  7.35812854e-01 ... -3.34903442e+00
  -6.56193443e-01 -5.86189130e-01]
 [ 6.13667523e-01 -2.95714327e-01  5.35570374e-01 ... -3.43197283e+00
  -3.06226916e-01 -3.42343365e-01]
 ...
 [ 3.58138434e-01 -8.13910911e-01  3.45177102e-01 ... -5.39539447e-03
   2.82869021e-02 -1.12276859e-01]
 [ 3.39737051e-01 -8.26269149e-01  3.43641258e-01 ... -5.39539447e-03
  -4.60715632e-03 -1.14986434e-01]
 [ 3.12696702e-01 -8.28133051e-01  3.34234213e-01 ... -5.39539447e-03
  -2.07165306e-01 -1.31583550e-01]]


In [61]:
assert(len(testseries_features)==len(test_data))
file = open('test_features.txt','wb')
pickle.dump(testseries_features,file)
file.close()

In [63]:
prediction = model.predict(testseries_features,batch_size=5000,verbose=1)
print(prediction.shape)
assert(len(prediction)==len(test_data))
print(prediction)

2345211/2345211 [==============================] - 5s 2us/step
(2345211, 1)
[[0.9841229 ]
 [1.        ]
 [0.02402413]
 ...
 [0.813516  ]
 [0.81009245]
 [0.773938  ]]


In [64]:
predict_flag = (prediction > 0.5).astype(int)
print(predict_flag)

[[1]
 [1]
 [0]
 ...
 [1]
 [1]
 [1]]


In [65]:
print(test_data[:, 2])

['02e99bd4f6cfb33f' '02e99bd4f6cfb33f' '02e99bd4f6cfb33f' ...
 '88cf3a776ba00e7c' '88cf3a776ba00e7c' '88cf3a776ba00e7c']


In [66]:
print(test_data[:, 0])

['1501475700' '1501475760' '1501475820' ... '1500825420' '1500825480'
 '1500825540']


In [67]:
assert(len(test_data[:,2])==len(predict_flag))

In [71]:
predict_flag = (np.ravel(prediction)>0.5).astype(int)
print(predict_flag)

[1 1 0 ... 1 1 1]


In [72]:
import pandas as pd
predict_df = pd.DataFrame({'KPI ID': test_data[:, 2], 
                         'timestamp': test_data[:, 0], 
                         'predict': predict_flag})
predict_df.to_csv('predict2.csv', index=False)